<a href="https://colab.research.google.com/github/mel-zheng/mel-zheng/blob/main/GPT_3_finetune_on_Shakespeare_(Hamlet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Dependencies

In [2]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json #copy kaggle API keys from the drive

In [5]:
!pip install --upgrade openai wandb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 492 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.8 MB 34.5 MB/s 
     |████████████████████████████████| 163 kB 43.9 MB/s 
     |████████████████████████████████| 181 kB 51.5 MB/s 
     |████████████████████████████████| 146 kB 50.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for openai: filename=openai-0.20.0-py3-none-any.whl size=54118 sha256=453342a4148480b543edf599151ac01f8a58713fad52a46c879c1d0b689fa93b
  Stored in directory: /root/.cache/pip/wheels/71/8d/9b/e28529ec53123e0279208f99148d4661232120d78cb866839b
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=4bef430a84f2539111529e47d6ef7a26dbdd7ab252baedaa25b04a9c424fc078
  Stored in direct

In [6]:
import openai
import wandb
from pathlib import Path
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import os

## Download data from Kaggle

In [7]:
!kaggle datasets download kingburrito666/shakespeare-plays -f Shakespeare_data.csv

  0% 0.00/2.78M [00:00<?, ?B/s]
100% 2.78M/2.78M [00:00<00:00, 200MB/s]


In [8]:
!unzip Shakespeare_data.csv.zip

Archive:  Shakespeare_data.csv.zip
  inflating: Shakespeare_data.csv    


## Process the raw kaggle data

In [9]:
!mkdir data

In [80]:
!mkdir data-hamlet

In [11]:
df_orig = pd.read_csv('Shakespeare_data.csv')
df_orig.head(5)

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [12]:
df_orig.Player.value_counts()

GLOUCESTER      1920
HAMLET          1582
IAGO            1161
FALSTAFF        1117
KING HENRY V    1086
                ... 
Haberdasher        1
NICHOLAS           1
JOSEPH             1
PHILIP             1
HORTENSIA          1
Name: Player, Length: 934, dtype: int64

In [27]:
df_hamlet = df_orig[df_orig.Player=='HAMLET']

In [28]:
df_hamlet.head(5)

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
32700,32701,Hamlet,9.0,1.2.66,HAMLET,"[Aside] A little more than kin, and less than..."
32702,32703,Hamlet,11.0,1.2.68,HAMLET,"Not so, my lord, I am too much i' the sun."
32709,32710,Hamlet,13.0,1.2.75,HAMLET,"Ay, madam, it is common."
32712,32713,Hamlet,15.0,1.2.78,HAMLET,"Seems, madam! nay it is, I know not 'seems.'"
32713,32714,Hamlet,15.0,1.2.79,HAMLET,"'Tis not alone my inky cloak, good mother,"


Drop NA

In [56]:
print(df_hamlet.shape)
df_hamlet = df_hamlet.dropna()
print(df_hamlet.shape)


(1582, 6)
(1503, 6)


PlayerLine are split into multiple rows. Recombine the lines into same line.

In [57]:
hamlet_lines = []
linenums = []
actscenes = []

num_prev = 0
act_prev=''
lines=""

for v in df_hamlet.values:
  act='.'.join(str(v[3]).split('.')[:2])
  num = v[2]
  line = v[-1]
  if num == num_prev and act_prev==act:
    lines = lines + ' ' + line
  else:
    if lines:
      hamlet_lines.append(lines)
      linenums.append(num_prev)
      actscenes.append(act_prev)
    lines = line
    num_prev = num
    act_prev = act

hamlet_lines.append(lines)
linenums.append(num_prev)
actscenes.append(act_prev)

In [58]:
df_hamlet_clean = pd.DataFrame(zip(linenums, actscenes, hamlet_lines), columns=['PlayerLinenumber','ActScene','PlayerLine'])

In [59]:
df_hamlet_clean.shape

(361, 3)

In [60]:
df_hamlet_clean.head(10)

,PlayerLinenumber,ActScene,PlayerLine
0,9.0,1.2,"[Aside] A little more than kin, and less than..."
1,11.0,1.2,"Not so, my lord, I am too much i' the sun."
2,13.0,1.2,"Ay, madam, it is common."
3,15.0,1.2,"Seems, madam! nay it is, I know not 'seems.' '..."
4,18.0,1.2,"I shall in all my best obey you, madam."
5,20.0,1.2,"O, that this too too solid flesh would melt Th..."
6,22.0,1.2,"I am glad to see you well: Horatio,--or I do f..."
7,24.0,1.2,"Sir, my good friend, I'll change that name wit..."
8,26.0,1.2,"I am very glad to see you. Good even, sir. But..."
9,28.0,1.2,"I would not hear your enemy say so, Nor shall ..."


Word Count ("wc")

In [61]:
df_hamlet_clean['wc'] = df_hamlet_clean['PlayerLine'].str.split().str.len()

In [64]:
df_hamlet_clean.wc.describe()

count    361.000000
mean      32.132964
std       52.545631
min        1.000000
25%        5.000000
50%       13.000000
75%       36.000000
max      461.000000
Name: wc, dtype: float64

Train with lines with more than 5 words

In [73]:
df_clean = df_hamlet_clean[df_hamlet_clean.wc>5]

In [74]:
df_clean.shape

(268, 4)

Create training dataset

In [76]:
df_clean = df_clean[['ActScene','PlayerLine']]
df_clean.columns=['prompt','completion']
df_clean.iloc[:,0]=""
df_clean.head(5)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


,prompt,completion
0,,"[Aside] A little more than kin, and less than..."
1,,"Not so, my lord, I am too much i' the sun."
3,,"Seems, madam! nay it is, I know not 'seems.' '..."
4,,"I shall in all my best obey you, madam."
5,,"O, that this too too solid flesh would melt Th..."


In [77]:
training_data_filename='Shakespeare_data_cleaned.csv'

In [78]:
df_clean.to_csv(f'data/{training_data_filename}', index=False)

In [79]:
df_clean.shape

(268, 2)

In [81]:
df_clean.to_csv(f'data-hamlet/{training_data_filename}', index=False)

## OpenAI API Key

In [1]:
# Enter credentials
%env OPENAI_API_KEY=[YOUR_OPENAI_API_KEY]

## Initiate W&B (Weights and Bias)

Sync jobs and dashboard to view/compare performances

You will need to request access for your API KEY.

In [83]:
project_name='GPT 3 for Generating Texts in Shakespeare tone (hamlet)'

run = wandb.init(project=project_name, job_type="dataset_preparation")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [84]:
run = wandb.init(project=project_name, entity="melzheng")

## Creat W&B Artifact

In [85]:
artifact = wandb.Artifact('data-hamlet', type='dataset')
artifact.add_dir('data-hamlet')
run.log_artifact(artifact) 

wandb: Adding directory to artifact (./data-hamlet)... Done. 0.1s


In [86]:
run = wandb.init(project=project_name)

artifact = run.use_artifact('data-hamlet:v0')
artifact_dir = artifact.download()+f"/{training_data_filename}"

In [87]:
#Shuffling the dataset

df = pd.read_csv(artifact_dir)
ds = df.sample(frac=1, random_state=0)


wandb.init(project=project_name, job_type="logging_dataset_as_table")
wandb.run.log({"Raw dataset" : wandb.Table(dataframe=ds)})

ds.to_csv(training_data_filename)
ds.head()

,prompt,completion
90,NaN,"It shall to the barber's, with your beard. Pri..."
260,NaN,"I dare not drink yet, madam, by and by."
76,NaN,"What, are they children? who maintains 'em? ho..."
83,NaN,"My lord, I have news to tell you. When Roscius..."
176,NaN,"I must to England, you know that?"


## OpenAI preprocess data

In [88]:
!openai tools fine_tunes.prepare_data -f Shakespeare_data_cleaned.csv

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 268 prompt-completion pairs
- The input file should contain exactly two columns/keys per row. Additional columns/keys present are: ['Unnamed: 0']
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Necessary] Your format `CSV` will be converted to `JSONL`
- [Necessary] Remove additional columns/keys: ['Unnamed: 0']
- [Recommended] Add a whitespace character to the beginning of the completion [Y/n]: Y
/usr/local/lib/python3.7/dist-packages/openai/validators.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [89]:
#The dataset has 500 pairs in total
with open('Shakespeare_data_cleaned_prepared.jsonl', 'r') as json_file:
    json_list = list(json_file)

num_data = len(json_list)
print("Total:", num_data)

val_part = 0.1 

val_amount = int(num_data * val_part)
print("Val data:", val_amount)
train_amount = num_data - val_amount 
print("Train data:", train_amount)

!head -n $train_amount Shakespeare_data_cleaned_prepared.jsonl > sh_train.jsonl
!tail -n $val_amount  Shakespeare_data_cleaned_prepared.jsonl > sh_valid.jsonl

Total: 268
Val data: 26
Train data: 242


In [90]:
wandb.finish()

## OpenAI Fine-tuning

In [91]:
'''define fine-tune params'''

model = 'curie'  # can be ada, babbage or curie
n_epochs = 4
batch_size = 4
learning_rate_multiplier = 0.1
prompt_loss_weight = 0.1
custom_modelname = 'Shakespeare-Hamlet-v0'

In [92]:
'''train'''

!openai api fine_tunes.create \
    -t sh_train.jsonl \
    -v sh_valid.jsonl \
    -m $model \
    --n_epochs $n_epochs \
    --batch_size $batch_size \
    --learning_rate_multiplier $learning_rate_multiplier \
    --prompt_loss_weight $prompt_loss_weight \
    --suffix $custom_modelname

Upload progress: 100% 59.8k/59.8k [00:00<00:00, 78.3Mit/s]
Uploaded file from sh_train.jsonl: file-Pkgwrn3gouwxi8VtsmHmdeV2
Upload progress: 100% 8.28k/8.28k [00:00<00:00, 10.9Mit/s]
Uploaded file from sh_valid.jsonl: file-Ex9W9ttcgAKNvU93MCYhEoEr
Created fine-tune: ft-cwLfwFSBz1fi2HP4NYk6umI8
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-11 14:13:49] Created fine-tune: ft-cwLfwFSBz1fi2HP4NYk6umI8
[2022-07-11 14:14:08] Fine-tune costs $0.16
[2022-07-11 14:14:09] Fine-tune enqueued. Queue number: 0
[2022-07-11 14:14:11] Fine-tune started
[2022-07-11 14:15:38] Completed epoch 1/4
[2022-07-11 14:16:12] Completed epoch 2/4
[2022-07-11 14:16:46] Completed epoch 3/4
[2022-07-11 14:17:19] Completed epoch 4/4
[2022-07-11 14:17:47] Uploaded model: curie:ft-personal:shakespeare-hamlet-v0-2022-07-11-14-17-45
[2022-07-11 14:17:49] Uploaded result file: file-2g6pVRur4zFSywxoPuqcqS99
[2022-07-11 14:17:50] Fine-tune succee

In [93]:
# sync fine-tune jobs to W&B
!openai wandb sync --project 'GPT 3 for Generating Texts in Shakespeare tone (hamlet)'

wandb: Currently logged in as: melzheng. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.21
wandb: Run data is saved locally in /content/wandb/run-20220711_142803-ft-FIwEP3IK7apKnBAFzYT1TrC0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-FIwEP3IK7apKnBAFzYT1TrC0
wandb: ⭐️ View project at https://wandb.ai/melzheng/GPT%203%20for%20Generating%20Texts%20in%20Shakespeare%20tone%20%28hamlet%29
wandb: 🚀 View run at https://wandb.ai/melzheng/GPT%203%20for%20Generating%20Texts%20in%20Shakespeare%20tone%20%28hamlet%29/runs/ft-FIwEP3IK7apKnBAFzYT1TrC0
File file-0WmhoqG2hyEsx7A0iNzvaFCw could not be retrieved. Make sure you are allowed to download training/validation files
File file-05Fz8bnpJhkRFtnkxpSLrV4G could not be retrieved. Make sure you are allowed to download training/validation files
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
w

## Log validation samples

In [94]:
# create eval job
run = wandb.init(project=project_name, job_type='eval')
entity = wandb.run.entity

wandb: Currently logged in as: melzheng. Use `wandb login --relogin` to force relogin


In [95]:
# choose a fine-tuned model
artifact_job = run.use_artifact(f'{entity}/{project_name}/fine_tune_details:latest', type='fine_tune_details')
artifact_job.metadata

{'created_at': 1657548829,
 'fine_tuned_model': 'curie:ft-personal:shakespeare-hamlet-v0-2022-07-11-14-17-45',
 'hyperparams': {'batch_size': 4,
  'learning_rate_multiplier': 0.1,
  'n_epochs': 4,
  'prompt_loss_weight': 0.1},
 'id': 'ft-cwLfwFSBz1fi2HP4NYk6umI8',
 'model': 'curie',
 'object': 'fine-tune',
 'organization_id': 'org-2ud1sYF9hh0kQ0RoB3CMcf09',
 'result_files': [{'bytes': 15739,
   'created_at': 1657549067,
   'filename': 'compiled_results.csv',
   'id': 'file-2g6pVRur4zFSywxoPuqcqS99',
   'object': 'file',
   'purpose': 'fine-tune-results',
   'status': 'processed',
   'status_details': None}],
 'status': 'succeeded',
 'training_files': [{'bytes': 59792,
   'created_at': 1657548825,
   'filename': 'sh_train.jsonl',
   'id': 'file-Pkgwrn3gouwxi8VtsmHmdeV2',
   'object': 'file',
   'purpose': 'fine-tune',
   'status': 'processed',
   'status_details': None}],
 'updated_at': 1657549071,
 'validation_files': [{'bytes': 8283,
   'created_at': 1657548828,
   'filename': 'sh_val

In [96]:
wandb.config.update({k:artifact_job.metadata[k] for k in ['fine_tuned_model', 'model', 'hyperparams']})

In [97]:
fine_tuned_model = artifact_job.metadata['fine_tuned_model']
fine_tuned_model

'curie:ft-personal:shakespeare-hamlet-v0-2022-07-11-14-17-45'

In [98]:
df = pd.read_json("sh_valid.jsonl", orient='records', lines=True)
df.head(5)

,prompt,completion
0,,"Come, for the third, Laertes: you but dally, ..."
1,,"He that plays the king shall be welcome, his ..."
2,,Do the boys carry it away?
3,,Angels and ministers of grace defend us! Be t...
4,,"Look here, upon this picture, and on this, Th..."


In [100]:
# inference on 30 validation examples.

n_samples = 30
df = df.iloc[:n_samples]

data = []

for _, row in tqdm(df.iterrows()):
    prompt = row['prompt']
    res = openai.Completion.create(model=fine_tuned_model, prompt=prompt, max_tokens=300, stop=[" END"])
    completion = res['choices'][0]['text']
    completion = completion[1:]       # remove initial space
    prompt = prompt[:-3]              # remove " ->"
    target = row['completion'][1:-4]  # remove initial space and "END"
    data.append([prompt, target, completion])

prediction_table = wandb.Table(columns=['prompt', 'target', 'completion'], data=data)
wandb.log({'predictions': prediction_table})

18it [01:25,  4.77s/it]


RateLimitError: ignored

In [101]:
wandb.finish() 

## Example Text Generation using fine-tuned model

option 1

In [114]:
!openai api completions.create -m curie:ft-personal:shakespeare-hamlet-v0-2022-07-11-14-17-45 -p 'the king would say the climate change is' -t 0.3 -M 32

the king would say the climate change is in his bones, and so it is: for, look you, these southern lands, that you call the Indies, have something in 'em that will change

In [113]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [117]:
model_name = 'curie:ft-personal:shakespeare-hamlet-v0-2022-07-11-14-17-45'

option 2

In [123]:
response = openai.Completion.create(model=model_name, prompt="the king might say the climate change is", 
                                    temperature=0.3, 
                                    max_tokens=64, 
                                    n=3,
                                    frequency_penalty=0.4, 
                                    presence_penalty=0, 
                                    logprobs=3)


## Open AI completion api parameters
Reference: https://beta.openai.com/docs/api-reference/models

__model*__


ID of the model to use. You can use the List models API to see all of your available models, or see our Model overview for descriptions of them.

__prompt__

Defaults to <|endoftext|>
The prompt(s) to generate completions for, encoded as a string, array of strings, array of tokens, or array of token arrays.

Note that <|endoftext|> is the document separator that the model sees during training, so if a prompt is not specified the model will generate as if from the beginning of a new document.

__suffix__

Defaults to null
The suffix that comes after a completion of inserted text.

__max_tokens__

Defaults to 16
The maximum number of tokens to generate in the completion.

The token count of your prompt plus max_tokens cannot exceed the model's context length. Most models have a context length of 2048 tokens (except for the newest models, which support 4096).

__temperature__

Defaults to 1
What sampling temperature to use. Higher values means the model will take more risks. Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

We generally recommend altering this or top_p but not both.

__top_p__

Defaults to 1
An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.

We generally recommend altering this or temperature but not both.

__n__

Defaults to 1
How many completions to generate for each prompt.

Note: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

__stream__

Defaults to false
Whether to stream back partial progress. If set, tokens will be sent as data-only server-sent events as they become available, with the stream terminated by a data: [DONE] message.

__logprobs__

Defaults to null
Include the log probabilities on the logprobs most likely tokens, as well the chosen tokens. For example, if logprobs is 5, the API will return a list of the 5 most likely tokens. The API will always return the logprob of the sampled token, so there may be up to logprobs+1 elements in the response.

The maximum value for logprobs is 5. If you need more than this, please contact support@openai.com and describe your use case.

__echo__

Defaults to false
Echo back the prompt in addition to the completion

__stop__
*string or array*

Defaults to null
Up to 4 sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

__presence_penalty__

Defaults to 0
Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

See more information about frequency and presence penalties.

__frequency_penalty__

Defaults to 0
Number between -2.0 and 2.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

See more information about frequency and presence penalties.

__best_of__

Defaults to 1
Generates best_of completions server-side and returns the "best" (the one with the highest log probability per token). Results cannot be streamed.

When used with n, best_of controls the number of candidate completions and n specifies how many to return – best_of must be greater than n.

Note: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

__logit_bias__

Defaults to null
Modify the likelihood of specified tokens appearing in the completion.

Accepts a json object that maps tokens (specified by their token ID in the GPT tokenizer) to an associated bias value from -100 to 100. You can use this tokenizer tool (which works for both GPT-2 and GPT-3) to convert text to token IDs. Mathematically, the bias is added to the logits generated by the model prior to sampling. The exact effect will vary per model, but values between -1 and 1 should decrease or increase likelihood of selection; values like -100 or 100 should result in a ban or exclusive selection of the relevant token.

As an example, you can pass {"50256": -100} to prevent the <|endoftext|> token from being generated.

In [124]:
response

<OpenAIObject text_completion id=cmpl-5SqYiKnBIahWWZQZKIvwkpFDNmUta at 0x7f9c1da0cdd0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          40,
          43,
          47,
          53,
          54,
          58,
          61,
          64,
          67,
          68,
          72,
          73,
          77,
          78,
          82,
          85,
          89,
          100,
          103,
          104,
          106,
          109,
          113,
          116,
          119,
          126,
          129,
          133,
          137,
          142,
          145,
          149,
          154,
          159,
          163,
          169,
          172,
          179,
          184,
          188,
          193,
          196,
          200,
          202,
          205,
          206,
          209,
          212,
          217,
          218,
          222,
          226,
          231,
          